## *Required Libraries*

In [35]:
import pandas as pd

## *Loading the Data*

In [36]:
train=pd.read_csv("Data/Train.csv")
test=pd.read_csv("Data/Test.csv")
waste=pd.read_csv("Data/waste_management.csv")
water=pd.read_csv("Data/water_sources.csv")
toilet=pd.read_csv("Data/toilets.csv")

In [37]:
train.head()

,ID,Total,Location,Category_Health_Facility_UUID,Disease,Month,Year,Transformed_Latitude,Transformed_Longitude
0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Dysentery,12,2022,-8.62982,68.23267
1,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Typhoid,12,2022,-8.62982,68.23267
2,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Diarrhea,12,2022,-8.62982,68.23267
3,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Diarrhea,12,2022,-8.62982,68.23267
4,ID_3a11929e-3317-476d-99f7-1bd9fb58f018_12_202...,0.0,ID_3a11929e-3317-476d-99f7-1bd9fb58f018,a9280aca-c872-46f5-ada7-4a7cc31cf6ec,Diarrhea,12,2022,-8.62982,68.23267


In [38]:
print("Train Data Shape -",train.shape)
print("Test Data Shape -",test.shape)
print("Waste Data Shape -",waste.shape)
print("Water Data Shape -",water.shape)
print("Toilet Data Shape -",toilet.shape)

Train Data Shape - (23848, 9)
Test Data Shape - (4872, 8)
Waste Data Shape - (5100, 43)
Water Data Shape - (10560, 43)
Toilet Data Shape - (10680, 43)


In [39]:
train.dtypes

ID                                object
Total                            float64
Location                          object
Category_Health_Facility_UUID     object
Disease                           object
Month                              int64
Year                               int64
Transformed_Latitude             float64
Transformed_Longitude            float64
dtype: object

## *Data Preprocessing*

In [40]:
train.isna().sum()

ID                               0
Total                            1
Location                         0
Category_Health_Facility_UUID    0
Disease                          0
Month                            0
Year                             0
Transformed_Latitude             0
Transformed_Longitude            0
dtype: int64

In [41]:
train.dropna(inplace=True)